# Guia práctica 2 

Utilizamos el dataset de kaggle : https://www.kaggle.com/datasets/andrewmvd/isic-2019/data


Instalar kaggle API y descargar el dataset mediante consola, escribir en consola :

pip install kaggle

kaggle datasets download -d andrewmvd/isic-2019

_________________________________________

Para establecer la configuracion de API de kaggle, es decir establecer donde se descarga el kaggle.json: 

In [3]:
import os

os.environ["KAGGLE_CONFIG_DIR"] = "C:/Users/elena/.kaggle"

# Descarga dataset

In [1]:
import kaggle

dataset_name = "andrewmvd/isic-2019"
kaggle.api.dataset_download_files(dataset_name, path="C:/Users/elena/Desktop/universidad/3º año/FSI/pythorch", unzip=True)


## Creacion pytorch dataset

In [5]:
import os
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

data_dir = "C:/Users/elena/Desktop/universidad/3º año/FSI/pythorch/ISIC_2019_Training_Input"  # The directory where the dataset was downloaded

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

dataset = ImageFolder(root=data_dir, transform=transform)

# You can create a DataLoader for batching and iterating over the dataset
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Now, you can use `dataloader` to iterate over the dataset in your PyTorch code.


## Definicion de una red convolucional de neuronas, modelo (CNN)


In [ ]:
import torch.nn as nn

class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 128 * 128, 64)  # Adjust input size based on your dataset
        self.fc2 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = x.view(-1, 16 * 128 * 128)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


## Establecer lod Data Loaders

In [23]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms
import os

# Define data transforms
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

print(os.getcwd())

# Update these paths to the correct directories
train_dir = r"C:\Users\elena\Desktop\universidad\3º año\FSI\pythorch\ISIC_2019_Training_Input\train"
test_dir = r"C:\Users\elena\Desktop\universidad\3º año\FSI\pythorch\ISIC_2019_Training_Input\test"

train_dataset = ImageFolder(
    root=train_dir,
    transform=transform
)

test_dataset = ImageFolder(
    root=test_dir,
    transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


C:\Users\elena\Desktop\universidad\3º año\FSI\pythorch


FileNotFoundError: Couldn't find any class folder in C:\Users\elena\Desktop\universidad\3º año\FSI\pythorch\ISIC_2019_Training_Input\train.

In [22]:
import os

# Manually navigate to each directory
try:
    os.chdir(r"C:\Users\elena\Desktop\universidad\3º año")
    print("Successfully changed to '3º año'")
    print("Listing contents:", os.listdir())
    os.chdir("FSI")
    print("Successfully changed to 'FSI'")
    print("Listing contents:", os.listdir())
    os.chdir("pythorch")
    print("Successfully changed to 'pythorch'")
    print("Listing contents:", os.listdir())
except Exception as e:
    print("An error occurred:", e)


Successfully changed to '3º año'
Listing contents: ['ASO', 'BD2', 'FSI', 'HORARIO PRIMER SEMESTRE.jpg', 'horario.jpeg', 'IS2']
Successfully changed to 'FSI'
Listing contents: ['P1', 'P1 - copia', 'P1.zip', 'pythorch', 'tabla.docx', 'Tabla.pdf', 'TABLA_P1.htm']
Successfully changed to 'pythorch'
Listing contents: ['.idea', 'ISIC_2019_Training_GroundTruth.csv', 'ISIC_2019_Training_Input', 'ISIC_2019_Training_Metadata.csv', 'model', 'notes', 'venv']
